In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg16 import VGG16,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [8]:
vgg16_base_model = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg16_base_model.output

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predict = Dense(10, activation='softmax')(x)

vgg16_model = Model(inputs=vgg16_base_model.input, outputs=predict)

In [5]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.2,
                                         zoom_range=0.2,
                                         rotation_range=15,
                                         height_shift_range=0.2,
                                         width_shift_range=0.2,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [9]:
for layer in vgg16_model.layers[:19]:
    layer.trainable = False

for i,layer in enumerate(vgg16_model.layers):
    print(i,layer.name,layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False
19 flatten_1 True
20 dense_1 True
21 batch_normalization_1 True
22 dropout_1 True
23 dense_2 True
24 batch_normalization_2 True
25 dropout_2 True
26 dense_3 True


In [11]:
vgg16_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-3),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=5)])

vgg16_model.save('vgg16_model_fc.h5')

Epoch 1/15
156/156 [==============================] - 346s 2s/step - loss: 1.9490 - acc: 0.3983 - val_loss: 0.8372 - val_acc: 0.7040
Epoch 2/15
156/156 [==============================] - 343s 2s/step - loss: 0.8677 - acc: 0.7075 - val_loss: 0.9587 - val_acc: 0.6594
Epoch 3/15
156/156 [==============================] - 342s 2s/step - loss: 0.6055 - acc: 0.7956 - val_loss: 1.0866 - val_acc: 0.6533
Epoch 4/15
156/156 [==============================] - 343s 2s/step - loss: 0.4961 - acc: 0.8360 - val_loss: 1.1221 - val_acc: 0.6494
Epoch 5/15
156/156 [==============================] - 341s 2s/step - loss: 0.4349 - acc: 0.8555 - val_loss: 1.1677 - val_acc: 0.6864
Epoch 6/15
156/156 [==============================] - 340s 2s/step - loss: 0.4115 - acc: 0.8662 - val_loss: 1.1821 - val_acc: 0.6616


In [6]:
# vgg16_model = load_model('vgg16_model_fc.h5')

# vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #,decay=1e-6,nesterov=True),
#                     #optimizer=optimizers.Adam(lr=1e-3),
# #                     optimizer=Adadelta(),
#                     metrics=['accuracy'])

# # fine-tune the model
# vgg16_model.fit_generator(
#                         train_generator,
#                         steps_per_epoch=nb_train_samples // batch_size,
#                         epochs=10,
#                         validation_data=validation_generator,
#                         validation_steps=nb_val_samples // batch_size,
#                         callbacks=[EarlyStopping(patience=5)])

# vgg16_model.save('vgg16_model_fc.h5')

Epoch 1/10
156/156 [==============================] - 501s 3s/step - loss: 0.4978 - acc: 0.8349 - val_loss: 1.6173 - val_acc: 0.5812
Epoch 2/10
156/156 [==============================] - 351s 2s/step - loss: 0.4881 - acc: 0.8382 - val_loss: 1.6326 - val_acc: 0.5745
Epoch 3/10
156/156 [==============================] - 342s 2s/step - loss: 0.4780 - acc: 0.8424 - val_loss: 1.5949 - val_acc: 0.5780
Epoch 4/10
156/156 [==============================] - 341s 2s/step - loss: 0.4750 - acc: 0.8413 - val_loss: 1.5597 - val_acc: 0.5902
Epoch 5/10
156/156 [==============================] - 341s 2s/step - loss: 0.4683 - acc: 0.8477 - val_loss: 1.4725 - val_acc: 0.6045
Epoch 6/10
156/156 [==============================] - 340s 2s/step - loss: 0.4564 - acc: 0.8469 - val_loss: 1.4280 - val_acc: 0.6115
Epoch 7/10
156/156 [==============================] - 339s 2s/step - loss: 0.4642 - acc: 0.8438 - val_loss: 1.4996 - val_acc: 0.5919
Epoch 8/10
156/156 [==============================] - 340s 2s/step - 

In [13]:
vgg16_model = load_model('vgg16_model_fc.h5')

for layer in vgg16_model.layers[:15]:
    layer.trainable = False
    
for layer in vgg16_model.layers[15:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
                    optimizer=optimizers.Adam(),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_model_15.h5')

Epoch 1/20
156/156 [==============================] - 347s 2s/step - loss: 0.5546 - acc: 0.8349 - val_loss: 0.8019 - val_acc: 0.8203
Epoch 2/20
156/156 [==============================] - 345s 2s/step - loss: 0.1958 - acc: 0.9400 - val_loss: 0.5082 - val_acc: 0.8911
Epoch 3/20
156/156 [==============================] - 345s 2s/step - loss: 0.0988 - acc: 0.9715 - val_loss: 0.4646 - val_acc: 0.8902
Epoch 4/20
156/156 [==============================] - 342s 2s/step - loss: 0.0644 - acc: 0.9808 - val_loss: 0.6298 - val_acc: 0.8297
Epoch 5/20
156/156 [==============================] - 343s 2s/step - loss: 0.0586 - acc: 0.9833 - val_loss: 1.2576 - val_acc: 0.7753
Epoch 6/20
156/156 [==============================] - 343s 2s/step - loss: 0.0598 - acc: 0.9817 - val_loss: 2.4103 - val_acc: 0.6250
Epoch 7/20
156/156 [==============================] - 345s 2s/step - loss: 0.0478 - acc: 0.9858 - val_loss: 0.4968 - val_acc: 0.8741
Epoch 8/20
156/156 [==============================] - 340s 2s/step - 

In [17]:
vgg16_model = load_model('vgg16_model_15.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True
    
vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])

vgg16_model.save('vgg16_model_11.h5')

Epoch 1/20
156/156 [==============================] - 354s 2s/step - loss: 0.0299 - acc: 0.9917 - val_loss: 0.4400 - val_acc: 0.8932
Epoch 2/20
156/156 [==============================] - 351s 2s/step - loss: 0.0263 - acc: 0.9936 - val_loss: 0.4189 - val_acc: 0.8859
Epoch 3/20
156/156 [==============================] - 346s 2s/step - loss: 0.0223 - acc: 0.9936 - val_loss: 0.4046 - val_acc: 0.8859
Epoch 4/20
156/156 [==============================] - 350s 2s/step - loss: 0.0197 - acc: 0.9947 - val_loss: 0.5079 - val_acc: 0.8619
Epoch 5/20
156/156 [==============================] - 351s 2s/step - loss: 0.0180 - acc: 0.9947 - val_loss: 0.5060 - val_acc: 0.8641
Epoch 6/20
156/156 [==============================] - 349s 2s/step - loss: 0.0152 - acc: 0.9960 - val_loss: 0.5075 - val_acc: 0.8545
Epoch 7/20
156/156 [==============================] - 346s 2s/step - loss: 0.0170 - acc: 0.9954 - val_loss: 0.5396 - val_acc: 0.8332
Epoch 8/20
156/156 [==============================] - 349s 2s/step - 

In [8]:
vgg16_model = load_model('vgg16_model_11.h5')

for layer in vgg16_model.layers[:11]:
    layer.trainable = False
    
for layer in vgg16_model.layers[11:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune')])
#                                    EarlyStopping(patience=6)])
vgg16_model.save('vgg16_final_model.h5')

Epoch 1/50
156/156 [==============================] - 505s 3s/step - loss: 0.0161 - acc: 0.9954 - val_loss: 0.4745 - val_acc: 0.8620
Epoch 2/50
156/156 [==============================] - 354s 2s/step - loss: 0.0162 - acc: 0.9957 - val_loss: 0.4456 - val_acc: 0.8672
Epoch 3/50
156/156 [==============================] - 347s 2s/step - loss: 0.0157 - acc: 0.9957 - val_loss: 0.4561 - val_acc: 0.8628
Epoch 4/50
156/156 [==============================] - 347s 2s/step - loss: 0.0156 - acc: 0.9957 - val_loss: 0.4847 - val_acc: 0.8576
Epoch 5/50
156/156 [==============================] - 348s 2s/step - loss: 0.0191 - acc: 0.9947 - val_loss: 0.4947 - val_acc: 0.8563
Epoch 6/50
156/156 [==============================] - 350s 2s/step - loss: 0.0151 - acc: 0.9963 - val_loss: 0.4819 - val_acc: 0.8619
Epoch 7/50
156/156 [==============================] - 343s 2s/step - loss: 0.0174 - acc: 0.9951 - val_loss: 0.5140 - val_acc: 0.8484
Epoch 8/50
156/156 [==============================] - 343s 2s/step - 

In [10]:
vgg16_model.save('vgg16_final_model.h5')

In [7]:
vgg16_model = load_model('vgg16_final_model.h5')

for layer in vgg16_model.layers[:7]:
    layer.trainable = False
    
for layer in vgg16_model.layers[7:]:
    layer.trainable = True

vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=15,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune'),
                                   EarlyStopping(patience=5)])


Epoch 1/15
156/156 [==============================] - 396s 3s/step - loss: 0.0718 - acc: 0.9778 - val_loss: 1.8490 - val_acc: 0.7292
Epoch 2/15
156/156 [==============================] - 403s 3s/step - loss: 0.0402 - acc: 0.9882 - val_loss: 0.3468 - val_acc: 0.9151
Epoch 3/15
156/156 [==============================] - 393s 3s/step - loss: 0.0337 - acc: 0.9899 - val_loss: 1.2201 - val_acc: 0.7169
Epoch 4/15
156/156 [==============================] - 393s 3s/step - loss: 0.0321 - acc: 0.9904 - val_loss: 0.4375 - val_acc: 0.8776
Epoch 5/15
156/156 [==============================] - 393s 3s/step - loss: 0.0273 - acc: 0.9910 - val_loss: 0.7590 - val_acc: 0.8149
Epoch 6/15
156/156 [==============================] - 394s 3s/step - loss: 0.0321 - acc: 0.9903 - val_loss: 1.9670 - val_acc: 0.7239
Epoch 7/15
156/156 [==============================] - 393s 3s/step - loss: 0.0258 - acc: 0.9913 - val_loss: 0.5251 - val_acc: 0.8985


In [8]:
vgg16_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg16_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg16_fine_tune'),
                                   CSVLogger('vgg16_fine_tune')])
#                                    EarlyStopping(patience=5)])

Epoch 1/10
156/156 [==============================] - 397s 3s/step - loss: 0.0349 - acc: 0.9910 - val_loss: 0.4889 - val_acc: 0.9045
Epoch 2/10
156/156 [==============================] - 395s 3s/step - loss: 0.0213 - acc: 0.9942 - val_loss: 0.3190 - val_acc: 0.9268
Epoch 3/10
156/156 [==============================] - 395s 3s/step - loss: 0.0219 - acc: 0.9939 - val_loss: 1.0300 - val_acc: 0.7901
Epoch 4/10
156/156 [==============================] - 393s 3s/step - loss: 0.0199 - acc: 0.9938 - val_loss: 0.6913 - val_acc: 0.8593
Epoch 5/10
156/156 [==============================] - 393s 3s/step - loss: 0.0228 - acc: 0.9929 - val_loss: 1.4624 - val_acc: 0.6847
Epoch 6/10
156/156 [==============================] - 393s 3s/step - loss: 0.0187 - acc: 0.9940 - val_loss: 1.5827 - val_acc: 0.7509
Epoch 7/10
156/156 [==============================] - 392s 3s/step - loss: 0.0171 - acc: 0.9947 - val_loss: 0.7462 - val_acc: 0.8841
Epoch 8/10
156/156 [==============================] - 392s 3s/step - 

In [9]:
vgg16_model.save('vgg16_final_7.h5')

In [11]:
vgg16_model = load_model('vgg16_final_model.h5')

test_model(vgg16_model,test_path,csv='vgg16_submission.csv')